<a href="https://colab.research.google.com/github/vverma022/traffic-ai/blob/main/traffic_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
import time
import cv2
import pandas as pd
from ultralytics import YOLO


In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
class_list = ['bicycle', 'car', 'motorcycle', 'bus','truck']

In [ ]:
tracker = cv2.TrackerCSRT_create()

In [ ]:
cap = cv2.VideoCapture('/trafficvideo.mp4')
if not cap.isOpened():
    print("Error: Could not open video.")

In [ ]:
previous_positions = {}
traffic_scores = []
bbox_id = 0
slow_speed_threshold = 2  # Speed threshold in pixels/frame
red_line_y = 198
blue_line_y = 268
offset = 7

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.resize(frame, (1020, 500))
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    car_list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
       if d < len(class_list):
            c = class_list[d]
            if c in ['car', 'bus', 'truck', 'bicycle', 'motorcycle', 'minivan']:
                car_list.append([x1, y1, x2, y2])

    # Track car speeds and calculate traffic score
    slow_cars_count = 0
    total_cars = 0

    for bbox in car_list:
        x3, y3, x4, y4 = bbox
        cx = int((x3 + x4) // 2)
        cy = int((y3 + y4) // 2)

        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

        # Calculate speed
        if bbox_id in previous_positions:
            prev_cx, prev_cy, prev_time = previous_positions[bbox_id]
            distance = ((cx - prev_cx) ** 2 + (cy - prev_cy) ** 2) ** 0.5
            time_elapsed = time.time() - prev_time
            speed = distance / time_elapsed

            # Check if the car is slow
            if speed < slow_speed_threshold:
                slow_cars_count += 1

        # Update previous positions
        previous_positions[bbox_id] = (cx, cy, time.time())
        total_cars += 1
        bbox_id += 1

    # Calculate traffic score
    if total_cars > 0:
        slow_percentage = (slow_cars_count / total_cars) * 100
        traffic_score = int(min(max((slow_percentage / 10), 1), 10))
    else:
        traffic_score = 1  # No cars detected, minimal traffic

    traffic_scores.append(traffic_score)

    # Display lines, car counts, and traffic score
    text_color = (255, 255, 255)  # white color for text
    red_color = (0, 0, 255)  # red color
    blue_color = (255, 0, 0)  # blue color
    green_color = (0, 255, 0)  # green color

    cv2.line(frame, (172, red_line_y), (774, red_line_y), red_color, 3)  # Red line
    cv2.putText(frame, 'red line', (172, red_line_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, blue_line_y), (927, blue_line_y), blue_color, 3)  # Blue line
    cv2.putText(frame, 'blue line', (8, blue_line_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, f'Traffic Score: {traffic_score}', (60, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow("Traffic Monitoring", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 21)